In [1]:
import nba_py
import datetime
import time
import nba_py.game
import nba_py.player
import nba_py.team
import pandas as pd
import numpy as np
import functools
import pymysql
from sqlalchemy import create_engine
%matplotlib inline

In [ ]:
conn = create_engine('mysql+pymysql://root:@localhost:3306/nba_stats')

player_list_18 = nba_py.player.PlayerList(season = '2017-18').info()['PERSON_ID']
print('Player list load complete!')

team_list = nba_py.team.TeamList().info()
team_id_dict = team_list[['TEAM_ID', 'ABBREVIATION']].set_index('ABBREVIATION').to_dict()['TEAM_ID']
print('Team list loaded!')

game_logs = pd.read_sql_table('game_logs', conn)
length_former = len(game_logs)
print(str(length_former) + ' game logs loaded.')

for p in player_list_18:
    for y in ['2014-15', '2015-16', '2016-17']:
        print(p, y, str(player_list_18[player_list_18 == p].index[0] + 1) + '/' + str(len(player_list_18)))
        game_logs = game_logs.append(nba_py.player.PlayerGameLogs(p, season = y).info())

'''
for p in player_list_18:
    game_logs = game_logs.append(nba_py.player.PlayerGameLogs(p, season = '2017-18').info())
'''
        
game_logs = game_logs.drop_duplicates(['Player_ID', 'Game_ID'])
length_new = len(game_logs)
print(str(length_new - length_former) + ' game logs append!')

game_logs['LOCATION'] = game_logs['MATCHUP'].apply(lambda x: 'AWAY' if '@' in x else 'HOME')
print('Game location assigned.')
game_logs['Oppo'] = game_logs['MATCHUP'].apply(lambda x: x[-3:])
print('Game opponent assigned.')
game_logs['Oppo_ID'] = game_logs['Oppo'].map(team_id_dict)
print('Game opponent id assigned.')

game_logs.to_sql('game_logs', conn, index = False, if_exists = 'replace')
print(str(length_new) + ' game logs commit complete!')